In [31]:
import numpy as np

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.nn import functional as F

from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import EarlyStopping

In [32]:
class BinaryData:
    def __init__(self, num_rows, num_cols):
        self.num_rows = num_rows
        self.num_cols = num_cols
        
        self.data = torch.randn(num_rows, num_cols)
        self.targets = torch.Tensor(np.random.choice([0, 1], num_rows))
    
    def __len__(self):
        return self.num_rows
    
    def __getitem__(self, idx):
        return self.data[idx, :], self.targets[idx]

In [35]:
class MyModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(50, 25)
        self.linear2 = nn.Linear(25, 1)
        
    def train_dataloader(self):
        return DataLoader(BinaryData(1000, 50), batch_size=16)
    
    def val_dataloader(self):
        return DataLoader(BinaryData(100, 50), batch_size=16)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return torch.sigmoid(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = F.binary_cross_entropy(y_hat, y)
        return {'loss': loss, 'train_loss': loss}
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = F.binary_cross_entropy(y_hat, y)
        return {'val_loss': loss}
    
    def training_epoch_end(self, outputs):
        loss_mean = torch.stack([x['train_loss'] for x in outputs]).mean()
        return {'train_loss': loss_mean}
    
    def validation_epoch_end(self, outputs):
        loss_mean = torch.stack([x['val_loss'] for x in outputs]).mean()
        return {'val_loss': loss_mean}

In [36]:
early_stop_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0.0,
    patience=80,
    verbose=True,
    mode="min"
)

trainer = Trainer(
    early_stop_callback=early_stop_callback,
    auto_select_gpus=True,
    max_epochs=200,
    terminate_on_nan=True,
    show_progress_bar=True,
    fast_dev_run=False,
    gpus=None
)

trainer.fit(MyModel())

INFO:lightning:GPU available: False, used: False
INFO:lightning:
  | Name    | Type   | Params
-------------------------------
0 | linear1 | Linear | 1 K   
1 | linear2 | Linear | 26    


INFO:lightning:Epoch 00041: early stopping
INFO:lightning:Epoch 00041: early stopping


1